[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/rycroft-group/math714/blob/main/e_iter_methods/iter_methods_demo.ipynb)

In [ ]:
# Necessity libraries
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

# Optional: a library for plotting with LaTeX-like 
# styles nicer formatted figures
# Warning: need to have LaTeX installed
import scienceplots
plt.style.use(['science'])

This code example varies the condition number of the SPD matrix and compares the number of iterations for the steepest descent and the conjugate gradient methods to find the solution.

In [ ]:
# Function to generate a 2x2 SPD matrix with given condition number
def make_spd_matrix(cond):
    # Eigenvalues: lambda1 = 1, lambda2 = cond
    eigvals = np.array([1, cond])
    # Random rotation
    theta = np.pi/4  # fixed rotation for demonstration
    Q = np.array([[np.cos(theta), -np.sin(theta)],
                  [np.sin(theta),  np.cos(theta)]])
    A = Q @ np.diag(eigvals) @ Q.T
    return A

# Steepest descent
def sd(A, F, x0, tol=1e-6, maxiter=100):
    x = x0.copy()
    r = F - A @ x
    xs = [x.copy()]
    for _ in range(maxiter):
        alpha = (r @ r) / (r @ A @ r)
        x = x + alpha * r
        r = F - A @ x
        xs.append(x.copy())
        if np.linalg.norm(r) < tol:
            break
    return x, xs

# Conjugate gradient
def cg(A, F, x0, tol=1e-6, maxiter=100):
    x = x0.copy()
    r = F - A @ x
    p = r.copy()
    xs = [x.copy()]
    for _ in range(maxiter):
        alpha = (r @ r) / (p @ A @ p)
        x = x + alpha * p
        r_new = r - alpha * (A @ p)
        xs.append(x.copy())
        if np.linalg.norm(r_new) < tol:
            break
        beta = (r_new @ r_new) / (r @ r)
        p = r_new + beta * p
        r = r_new
    return x, xs

# Function to compare SD and CG methods
def compare_methods(cond=10.0):
    # Construct matrix equation Au = F
    A = make_spd_matrix(cond)
    F = np.array([1, 2])

    # Initial guess
    x_sd = np.zeros(2)
    x_cg = np.zeros(2)

    # Exact solution
    x_star = np.linalg.solve(A, F)

    # Numerical solutions
    x_sd, xs_sd_list = sd(A, F, x_sd)
    x_cg, xs_cg_list = cg(A, F, x_cg)
    xs_sd = np.array(xs_sd_list)
    xs_cg = np.array(xs_cg_list)
    
    # Plot contours
    x1 = np.linspace(-1, 3, 200)
    x2 = np.linspace(-1, 3, 200)
    X1, X2 = np.meshgrid(x1, x2)
    Z = 0.5 * (A[0,0]*X1**2 + 2*A[0,1]*X1*X2 + A[1,1]*X2**2) - F[0]*X1 - F[1]*X2
    
    plt.figure(figsize=(8,6))
    plt.contour(X1, X2, Z, levels=30, cmap='viridis')
    
    # Plot SD iterates
    plt.plot(xs_sd[:,0], xs_sd[:,1], 'r-', label='Steepest Descent')
    for i in range(len(xs_sd)-1):
        plt.arrow(xs_sd[i,0], xs_sd[i,1], xs_sd[i+1,0]-xs_sd[i,0], xs_sd[i+1,1]-xs_sd[i,1],
                  head_width=0.0, color='red', alpha=0.5)
    
    # Plot CG iterates
    plt.plot(xs_cg[:,0], xs_cg[:,1], 'bo-', label='Conjugate Gradient')
    for i in range(len(xs_cg)-1):
        plt.arrow(xs_cg[i,0], xs_cg[i,1], xs_cg[i+1,0]-xs_cg[i,0], xs_cg[i+1,1]-xs_cg[i,1],
                  head_width=0.05, color='blue', alpha=0.8)
    
    # Exact solution
    plt.plot(x_star[0], x_star[1], 'kx', markersize=10, label='Exact solution')
    
    plt.xlabel('$x_1$')
    plt.ylabel('$x_2$')
    plt.title(f'Steepest descent v.s. conjugate gradient (cond(A)={cond:.1f})')
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
# Interactive widget to vary condition number
interact(compare_methods, cond=FloatSlider(value=10.0, min=1.0, max=100.0, step=5.0, description='Condition Number'));